In [1]:
import pyscf
import pyscf.tools

from orbitalpartitioning import *
from pyscf.gto.basis import parse_gaussian

/home/arnabbachhar/.local/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Fe -1.67785607 -0.00052233 0.06475932
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
 """

In [9]:
# basis = "def2-tzvp"
basis = {
        'Cl' : parse_gaussian.load('ano-rcc-vdz.gbs','Cl'),
        'O' : parse_gaussian.load('ano-rcc-vdz.gbs','O'),
        'Fe' : parse_gaussian.load('ano-rcc-vdz.gbs','Fe')
    }
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()

print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.ROHF(pymol).newton()
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.run()

symmetry:  C2


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> Newton solver flags ********
SCF tol = 1e-08
conv_tol_grad = 1e-05
max. SCF cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/arnabbachhar/workspace/FermiCG-data/bimetallics/fe2_morokuma/26_3d4d_2p3p_3d4d/tmpznpd042e
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 518 MB)


HOMO (A) = 0.319251543683352  LUMO (B) = 0.31992426031573
Initial guess E= -5333.33250659904  |g|= 7.12818
macro= 0  E= -5343.40594745205  delta_E= -10.0734  |g|= 2.22533  3 KF 15 JK
macro= 1  E= -5345.55695553284  delta_E= -2.15101  |g|= 0.523879  3 KF 16 JK
macro= 2  E= -5345.99445383373  delta_E= -0.437498  |g|= 0.240282  3 KF 17 JK
macro= 3  E= -5346.0225261253  delta_E= -0.0280723  |g|= 0.0777355  3 KF 13 JK
macro= 4  E= -5346.02971360072  delta_E= -0.00718748  |g|= 0.0682769  2 KF 8 JK
macro= 5  E= -5346.03694866058  delta_E= -0.00723506  |g|= 0.027192  3 KF 13 JK
macro= 6  E= -5346.03768218459  delta_E= -0.000733524  |g|= 0.00656955  3 KF 15 JK
macro= 7  E= -5346.03774105611  delta_E= -5.88715e-05  |g|= 0.00353509  2 KF 9 JK
macro= 8  E= -5346.03776965305  delta_E= -2.85969e-05  |g|= 0.00215272  2 KF 10 JK
macro= 9  E= -5346.03778184729  delta_E= -1.21942e-05  |g|= 0.00129125  2 KF 9 JK
macro= 10  E= -5346.03778792236  delta_E= -6.07507e-06  |g|= 0.000846665  2 KF 10 JK
macro= 1

SecondOrderROHF-SymAdaptedROHF-ROHF-RHF object of <class 'pyscf.soscf.newton_ah.newton.<locals>.SecondOrderROHF'>

In [10]:
#
#   Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
Cdocc = mf.mo_coeff[:,mf.mo_occ==2]
Csing = mf.mo_coeff[:,mf.mo_occ==1]
Cvirt = mf.mo_coeff[:,mf.mo_occ==0]
ndocc = Cdocc.shape[1]
nsing = Csing.shape[1]
nvirt = Cvirt.shape[1]

In [11]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d","4d"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("2p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3d","4d"):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)

[[17, 18, 19, 20, 21, 22, 23, 24, 25, 26], [30, 31, 32], [53, 54, 55, 56, 57, 58, 59, 60, 61, 62]]


In [12]:
#   Get full active space
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning_nonorth((Cdocc, Csing, Cvirt), full, S)
# assert(Cerr.shape[1] == 0)

 In svd_subspace_partitioning_nonorth
 Partition  141 orbitals into a total of   23 orbitals
            Index   Sing. Val. Space       
                0   0.99874844            0*
                1   0.99851411            0*
                2   0.99674784            0*
                3   0.98754592            1*
                4   0.98699137            1*
                5   0.98387280            1*
                6   0.98329237            1*
                7   0.97830876            2*
                8   0.95986667            1*
                9   0.95938943            1*
               10   0.94455146            1*
               11   0.93629649            1*
               12   0.93413709            2*
               13   0.93044503            2*
               14   0.90059815            2*
               15   0.89833231            2*
               16   0.89266101            1*
               17   0.86468087            2*
               18   0.85922970            2*
        

In [13]:
#   Split active space into fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning_nonorth((Oact, Sact, Vact), f, S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

print(" sing vals of active space: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Cact_23.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv_23.molden", Cenv)
for i in range(len(frags)):
    di = Cfrags[i] @ Cfrags[i].T
    pyscf.tools.cubegen.density(mf.mol, 'fragden_{:02d}.cube'.format(i+1), di)


print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 Fragment:  [17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
 In svd_subspace_partitioning_nonorth
 Partition   23 orbitals into a total of   10 orbitals
            Index   Sing. Val. Space       
                0   0.97750534            1*
                1   0.97657988            1*
                2   0.95482778            1*
                3   0.93523138            1*
                4   0.88716527            2*
                5   0.87613326            2*
                6   0.86386513            2*
                7   0.84458743            2*
                8   0.80632404            2*
                9   0.62819816            1*
               10   0.56825731            0
               11   0.36274078            0
               12   0.35854622            2
               13   0.35565885            0
               14   0.03993740            1
               15   0.03287026            1
               16   0.01041775            1
               17   0.00592353            2
       

 sing vals of active space:  [1.00000001e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 9.99999996e-01 2.66480835e-16]

WARN: orbitals [ 0  1  2  3  4  5  6  7  8  9 13 14 15 16 17 18 19 20 21 22] not symmetrized, norm = [0.50011255 0.50006625 0.50140937 0.50365201 0.97958662 0.5
 0.5        0.5        0.5        0.5        0.50011255 0.50006625
 0.50140936 0.50365201 0.97958663 0.5        0.5        0.5
 0.5        0.5       ]


WARN: orbitals [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67
 68 69 70 71 72] not symmetrized, norm = [0.58946247 0.7332401  0.55782872 0.66716252 0.53178081 0.61046275
 0.61150121 0.51218708 0.60663759 0

In [14]:
#   Make integrals
#
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

In [15]:

np.save("ints_h0_23", h0)
np.save("ints_h1_23", h1)
np.save("ints_h2_23", h2)
np.save("mo_coeffs_23", Cact)
np.save("overlap_mat_23", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa_23", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb_23", Cact.T @ S @ Pb @ S @ Cact)